# Reto: competición de sistemas de clasificación

En este reto aplicaremos regresión logística a la tarea de clasificación [*california-housing*](https://scikit-learn.org/1.5/datasets/real_world.html#california-housing-dataset).

El objetivo de esta tarea es clasificar, con mínimo error, diferentes distritos del estado de California (EEUU) en 6 clases diferentes correspondientes a rangos de precios medios de vivienda, utilizando un conjunto de D=8 características numéricas.

## 1. Obtención del corpus y partición de datos

Ejecuta el siguiente bloque de código, en el que importamos las librerías necesarias, definimos constantes, obtenemos el dataset, barajamos (con una semilla concreta e inalterable) y particionamos los datos en train y test (con una proporción concreta e inalterable).

**MUY IMPORTANTE: NO MODIFICAR ESTE BLOQUE DE CÓDIGO, SOLO EJECUTAR.**

In [19]:
### NO MODIFIQUES ESTE BLOQUE DE CÓDIGO; SOLO EJECÚTALO ###

import warnings; warnings.filterwarnings("ignore"); import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

RANDOM_SEED = 22
TEST_SIZE = 0.2

### NO MODIFIQUES ESTE BLOQUE DE CÓDIGO; SOLO EJECÚTALO ###

corp = fetch_california_housing()
X = corp.data.astype(np.float16) # muestras
y = corp.target.astype(np.uint)  # etiquetas de clase

### NO MODIFIQUES ESTE BLOQUE DE CÓDIGO; SOLO EJECÚTALO ###

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED)
D = X_train.shape[1]; C=np.unique(y_train).size; Ntr = X_train.shape[0]; Nte = X_test.shape[0];  

print("******** INFORMACIÓN BÁSICA DEL DATASET **********")
print(f"D={D}, C={C}, N_train={Ntr}, N_test={Nte}")

### NO MODIFIQUES ESTE BLOQUE DE CÓDIGO; SOLO EJECÚTALO ###

******** INFORMACIÓN BÁSICA DEL DATASET **********
D=8, C=6, N_train=16512, N_test=4128


## 2. Entrenamiento y evaluación del sistema base (baseline)

A continuación podéis obtener una tasa de error de referencia, obtenido con un clasificador de regresión logística entrenado con `tol=0.01`, `C=1`, y `max_iter=10`. Este será nuestro sistema de clasificación base (baseline). 

Vuestro objetivo será mejorar (lo máximo posible) la tasa de error obtenida con este sistema.

In [20]:
# NOTA IMPORTANTE: SIEMPRE USAREMOS random_state=RANDOM_SEED
clf = LogisticRegression(random_state=RANDOM_SEED, C=1, tol=0.01, max_iter=10).fit(X_train, y_train)
err_test = (1 - accuracy_score(y_test, clf.predict(X_test)))*100
print(f'Error de clasificación en test (baseline): {err_test:5.1f}%')

Error de clasificación en test (baseline):  59.9%


## 3. Exploración/Optimización de hiperparámetros

Realiza una exploración y ajuste de los hiperparámetros tolerancia (`tol`), escalado del factor de regularización (`C`), y número de iteraciones máximas (`max_iter`), para minimizar la tasa de error de clasificación en test. 

Reporta los resultados de los experimentos en una o varias tablas que muestren los valores de los tres hiperparámetros mencionados, además de las tasas de error de clasificación en train y test. 

Introduce a continuación el código que hayas utilizado para realizar esta exploración/optimización, y asegúrate que el cuaderno conserva la salida de la ejecución de dicho código. Crea celdas de código adicionales si lo necesitas.

**IMPORTANTE: usa el parámetro `random_state=RANDOM_SEED` en `LogisticRegression()`**.

In [21]:
#### COMPLETAR!

results = []

C_values = [0.01, 0.1, 1, 10, 100]
tol_values = [1e-2, 1e-3, 1e-4]
max_iter_values = [100, 500, 1000, 5000]

for C in C_values:
    for tol in tol_values:
        for max_iter in max_iter_values:

            clf = LogisticRegression(
                C=C,
                tol=tol,
                max_iter=max_iter,
                random_state=RANDOM_SEED
            )

            clf.fit(X_train, y_train)

            y_train_pred = clf.predict(X_train)
            y_test_pred = clf.predict(X_test)

            err_train = (1 - accuracy_score(y_train, y_train_pred)) * 100
            err_test = (1 - accuracy_score(y_test, y_test_pred)) * 100

            results.append([C, tol, max_iter, err_train, err_test])

# Mostrar resultados ordenados por menor error en test
import pandas as pd

df_results = pd.DataFrame(
    results,
    columns=["C", "tol", "max_iter", "Error train (%)", "Error test (%)"]
)

df_results = df_results.sort_values(by="Error test (%)")
df_results

,C,tol,max_iter,Error train (%),Error test (%)
43,10.00,0.0010,5000,44.295058,45.058140
47,10.00,0.0001,5000,44.295058,45.058140
39,10.00,0.0100,5000,44.295058,45.058140
27,1.00,0.0100,5000,44.016473,45.082364
35,1.00,0.0001,5000,44.016473,45.082364
31,1.00,0.0010,5000,44.016473,45.082364
59,100.00,0.0001,5000,44.349564,45.300388
55,100.00,0.0010,5000,44.349564,45.300388
51,100.00,0.0100,5000,44.349564,45.300388
23,0.10,0.0001,5000,44.694767,45.494186


## 4. Determinación de los hiperparámetros óptimos y tasas de error

Por último, **modifica** la siguiente celda para que indique cuáles son los valores óptimos de los tres hiperparámetros `C`, `tol` y `max_iter`, así como las correspondientes tasas de error obtenidas en los conjuntos de entrenamiento y test. 

- **Factor de regularización (C):** 10
- **Tolerancia (tol):** 0.01
- **Número máximo de iteraciones (max_iter):** 5000
- **Tasa de error (%) obtenida en train:** 44.30
- **Tasa de error (%) obtenida en test:** 45.06

Nota: los valores de `C`, `tol` y `max_iter` que indiques aquí son los que usarás para participar en la evaluación final.